# yt_dlp

参考サイト：
[https://masayoshi-9a7ee.hatenablog.com/entry/2021/11/06/112639](https://masayoshi-9a7ee.hatenablog.com/entry/2021/11/06/112639)

In [ ]:
! pip install -U yt_dlp

# 【旧版】

In [ ]:
from yt_dlp import YoutubeDL
from pathlib import Path

output_dir = Path("./output/youtube/")
if not output_dir.exists():
  output_dir.mkdir(parents=True)

youtube_video_URL_list = [
  "https://www.youtube.com/watch?v=XXXXXXXXXXX", # single video
  # "https://www.youtube.com/watch?v=XXXXXXXXXXX", # playlist
]

ydl_opts = {
            'format': 'bestvideo/best',
            'outtmpl': f'{str(output_dir)}/%(uploader)s/%(upload_date)s_%(title)s_%(id)s.%(ext)s',
            }

with YoutubeDL(ydl_opts) as ydl:
    result = ydl.download(youtube_video_URL_list)

# 【新版】

In [1]:
import sys
import yt_dlp
from tqdm import tqdm
from pathlib import Path

def download_youtube_video(url, output_dir = Path("./output/youtube/")):
    """
    指定した YouTube 動画を H.264 + AAC の MP4 ファイルとしてダウンロードする。
    """
    
    if not output_dir.exists():
        output_dir.mkdir(parents=True)

    # tqdmのインスタンスを先に作成
    pbar = tqdm(
        total=0, 
        unit='B', 
        unit_scale=True, 
        desc='Downloading', 
        leave=True
    )

    def download_progress_hook(d):

        if d['status'] == 'downloading':
            # 合計サイズ (Byte) がわかっていれば反映
            if d.get('total_bytes'):
                pbar.total = d['total_bytes']
            elif d.get('total_bytes_estimate'):
                pbar.total = d['total_bytes_estimate']

            # ダウンロード済みサイズ (Byte) を反映
            if d.get('downloaded_bytes'):
                pbar.n = d['downloaded_bytes']
                pbar.refresh()

            pbar.set_postfix_str(f"title=\"{d['info_dict']['title']}\", ext={d['info_dict']['ext']}")
            
        elif d['status'] == 'finished':
            print()
            pbar.refresh()

    # yt_dlp に渡すためのカスタムlogger（標準出力へのログを抑制）
    class TqdmLogger(object):
        def debug(self, msg):
            pass
        def warning(self, msg):
            pass
        def error(self, msg):
            sys.stderr.write(msg + '\n')

    # ダウンロードオプション
    ydl_opts = {
        'format': 'bestvideo[ext=mp4][vcodec^=avc1]+bestaudio[ext=m4a]/mp4',
        'outtmpl': f'{str(output_dir)}/%(uploader)s/%(upload_date)s_%(title)s_%(id)s.%(ext)s',
        'merge_output_format': 'mp4',
        'logger': TqdmLogger(),
        'quiet': True,
        'no_warnings': True,
        'progress_hooks': [download_progress_hook]
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])


In [ ]:
output_dir = Path("./output/youtube/")
youtube_video_url = "https://www.youtube.com/watch?v=XXXXXXXXXXX"
download_youtube_video(youtube_video_url, output_dir)